In [1]:
!git clone https://github.com/jaywalnut310/vits.git

Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81 (from 1)
Receiving objects: 100% (81/81), 3.33 MiB | 11.57 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [2]:
%cd vits

/content/vits


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 54.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
%cd vits  # If you restart runtime
%cd monotonic_align

[Errno 2] No such file or directory: 'vits # If you restart runtime'
/content/vits
/content/vits/monotonic_align


In [5]:
!python setup.py build_ext --inplace

Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: core.pyx:7:5: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returned.
performance hint: core.pyx:38:6: Exception check on 'maximum_path_c' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_c' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. 

In [6]:
!sudo apt-get install espeak -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,382 kB of archives.
After this operation, 3,178 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak-data amd64 1.48.15+dfsg-3 [1,085 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libespeak1 amd64 1.48.15+dfsg-3 [156 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak amd64 1.48.15+dfsg-3 [64.2 kB]
Fetched 1,382 kB in 1s (1,379 

In [7]:
%cd ..

/content/vits


In [8]:
!pip install gdown

In [9]:
!gdown 'https://drive.google.com/uc?id=1q86w74Ygw2hNzYP9cWkeClGT5X25PvBT'

Downloading...
From (original): https://drive.google.com/uc?id=1q86w74Ygw2hNzYP9cWkeClGT5X25PvBT
From (redirected): https://drive.google.com/uc?id=1q86w74Ygw2hNzYP9cWkeClGT5X25PvBT&confirm=t&uuid=9bf8804c-3c3d-49e8-acf1-41510c6ed105
To: /content/vits/pretrained_ljs.pth
100% 146M/146M [00:00<00:00, 164MB/s]


In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

ModuleNotFoundError: No module named 'unidecode'

In [ ]:
hps = utils.get_hparams_from_file("./configs/ljs_base.json")

In [ ]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint("pretrained_ljs.pth", net_g, None)

In [ ]:
stn_tst = get_text("We propose VITS, Conditional Variational Autoencoder with Adversarial Learning for End-to-End Text-to-Speech.", hps)
with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))

## Multi Speaker

In [ ]:
!gdown 'https://drive.google.com/uc?id=11aHOlhnxzjpdWDpsz1vFDCzbeEfoIxru'

In [ ]:
hps_ms = utils.get_hparams_from_file("./configs/vctk_base.json")

In [ ]:
net_g_ms = SynthesizerTrn(
    len(symbols),
    hps_ms.data.filter_length // 2 + 1,
    hps_ms.train.segment_size // hps.data.hop_length,
    n_speakers=hps_ms.data.n_speakers,
    **hps_ms.model)
_ = net_g.eval()

_ = utils.load_checkpoint("pretrained_vctk.pth", net_g_ms, None)

In [ ]:
sid = torch.LongTensor([4]) # speaker identity
stn_tst = get_text("We propose VITS, Conditional Variational Autoencoder with Adversarial Learning for End-to-End Text-to-Speech.", hps_ms)

with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g_ms.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.float().numpy()
ipd.display(ipd.Audio(audio, rate=hps_ms.data.sampling_rate))

## Voice Conversion

In [ ]:
!wget https://jaywalnut310.github.io/vits-demo/wavs/vc_01_01.wav

In [ ]:
from mel_processing import spectrogram_torch
from utils import load_wav_to_torch

audio, sampling_rate = load_wav_to_torch("./vc_01_01.wav")

y = audio / hps_ms.data.max_wav_value
y = y.unsqueeze(0)

spec = spectrogram_torch(y, hps_ms.data.filter_length,
    hps_ms.data.sampling_rate, hps_ms.data.hop_length, hps_ms.data.win_length,
    center=False)
spec_lengths = torch.LongTensor([spec.size(-1)])
sid_src = torch.LongTensor([81])

In [ ]:
with torch.no_grad():
    sid_tgt1 = torch.LongTensor([77])
    sid_tgt2 = torch.LongTensor([14])
    sid_tgt3 = torch.LongTensor([1])
    sid_tgt4 = torch.LongTensor([17])
    audio1 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.float().numpy()
    audio2 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.float().numpy()
    audio3 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.float().numpy()
    audio4 = net_g_ms.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt4)[0][0,0].data.float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].numpy(), rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps_ms.data.sampling_rate))
print("Converted SID: %d" % sid_tgt4.item())
ipd.display(ipd.Audio(audio4, rate=hps_ms.data.sampling_rate))

Original SID: 81


Converted SID: 77


Converted SID: 14


Converted SID: 1


Converted SID: 17
